***********************



THIS CREATES THE CSV WITH THE RACE LABEL
**************************


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [6]:
df = pd.read_csv('labeling-responses.csv')

In [7]:
wgbh_urls = []
race_data = []

for index, row in df.iterrows():
    # Get the URL from the column 'Paste the link of the article read.'
    url = row['Paste the link of the article read.']
    if 'wgbh.org' in url:
        # If 'wgbh.org' is in the URL, append the URL to the 'wgbh_urls' list
        wgbh_urls.append(url)

        # Append the race data from the column 'What race of people was discussed in the article?' to the 'race_data' list
        race_data.append(row['What race of people was discussed in the article?'])

['https://www.wgbh.org/news/national/2023-03-18/why-car-prices-are-still-so-high-and-why-they-are-unlikely-to-fall-anytime-soon', 'https://www.wgbh.org/news/politics/2023-03-08/boston-city-council-approves-wus-rent-control-and-city-planning-proposals', 'https://www.wgbh.org/news/local/2023-06-06/we-have-a-crisis-leaders-look-to-address-root-of-gun-violence-in-boston', 'https://www.wgbh.org/news/commentary/2023-04-03/it-should-be-so-much-easier-to-cancel-unwanted-subscriptions', 'https://www.wgbh.org/news/local/2023-03-17/basic-black-how-will-bostons-task-force-on-reparations-confront-generational-trauma-caused-by-enslavement', 'https://www.wgbh.org/news/national-news/2023/04/28/airbnb-let-its-workers-live-and-work-anywhere-spoiler-theyre-loving-it', 'https://www.wgbh.org/news/local/2023-07-19/paul-revere-never-owned-slaves-but-he-still-benefited-from-the-slave-trade', 'https://www.wgbh.org/news/local/2023-03-24/10-year-old-tanoshi-inomata-is-spelling-his-way-from-boston-to-d-c', 'https

In [10]:

# A function to scrape articles from 'wgbh.org'
def scrape_wgbh_articles_2(url, race):
    # Send a GET request to the provided URL
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the article title and article body
        article_title = soup.find(class_ = 'heading heading-1')
        if article_title:
            article_title = article_title.text.strip()
        else:
            article_title = "Title not found"
        
        article_body = soup.find(class_ = 'articleBody rich-text')
        if article_body:
            # Remove any 'enhancements' from the body
            enhancements = article_body.find_all(class_='Enhancement')
            for enhancement in enhancements:
                enhancement.decompose()

            # Get the text of the article body
            article_text = article_body.get_text(separator="", strip=True)
        else:
            article_text = "Article text not found"

        # Return a dictionary with the scraped data
        return {
            'url': url,
            'title': article_title,
            'text': article_text,
            'race_label': race  # Add the 'race' data to the returned dictionary
        }

    # If the GET request was not successful, print an error message and return None
    else:
        print(f"Failed to retrieve content from {url}")
        return None

scraped_articles = []

# Iterate over each URL and corresponding race data
for url, race in zip(wgbh_urls, race_data):
    # Call the 'scrape_wgbh_articles' function to scrape the article data
    article_info = scrape_wgbh_articles_2(url, race)
    if article_info:  # Check if the article info was successfully retrieved
        scraped_articles.append(article_info)
    
    time.sleep(1)  # Pause for 1 second between requests to be respectful to the server

# Example output
# Print the title, URL, and race label of the first 5 scraped articles to check the output
for article in scraped_articles[:5]: 
    print(article['title'], article['url'], article['race_label'])

Why car prices are still so high — and why they are unlikely to fall anytime soon https://www.wgbh.org/news/national/2023-03-18/why-car-prices-are-still-so-high-and-why-they-are-unlikely-to-fall-anytime-soon nan
Boston City Council approves Wu's rent control and city planning proposals https://www.wgbh.org/news/politics/2023-03-08/boston-city-council-approves-wus-rent-control-and-city-planning-proposals nan
‘We have a crisis’: Leaders look to address root of gun violence in Boston https://www.wgbh.org/news/local/2023-06-06/we-have-a-crisis-leaders-look-to-address-root-of-gun-violence-in-boston nan
It should be so much easier to cancel unwanted subscriptions https://www.wgbh.org/news/commentary/2023-04-03/it-should-be-so-much-easier-to-cancel-unwanted-subscriptions nan
Basic Black: How will Boston’s task force on reparations confront generational trauma caused by enslavement? https://www.wgbh.org/news/local/2023-03-17/basic-black-how-will-bostons-task-force-on-reparations-confront-gener

In [11]:
import csv 

csv_file_name = 'scraped_wgbh_articles_attempt2.csv'

headers = ['title', 'text', 'url', 'race_label']

with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    
    # Write the header to the CSV file
    writer.writeheader()
    
    # Write the scraped article data to the CSV file
    for article in scraped_articles:
        writer.writerow(article)

print(f"Data has been written to {csv_file_name}")

Data has been written to scraped_wgbh_articles_attempt2.csv
